# [Projet 3: Développez un moteur de recommandations de films](https://openclassrooms.com/projects/developpez-un-moteur-de-recommandations-de-films)

Link to raw data: [here](https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Moteur+de+recommandation+de+films/imdb-5000-movie-dataset.zip).

In [1]:
import os
HOME = os.path.expanduser('~/')
HOST = os.uname()[1]
if HOST == 'Arthurs-MacBook-Pro.local':
    os.chdir(HOME+'/Documents/GitHub/OCDataSciencePath/Project3/')    # @home
elif HOST == 'Sirius.local':
    os.chdir(HOME+'Perso/GitHub/OCDataSciencePath/Project3/')         # @L2
else:
    raise ValueError('unknown host: {}'.format(HOST))
    
import numpy as np
import pandas as pd

In [2]:
if HOST == 'Arthurs-MacBook-Pro.local':
    pathToData = HOME+'xxx'    # @home
elif HOST == 'Sirius.local':
    pathToData = HOME+'Downloads/movie_metadata.csv'         # @L2
else:
    raise ValueError('unknown host: {}'.format(HOST))
    
df = pd.read_csv(pathToData)

In [8]:
n,p = df.shape
print(n,p)
df.sample(5)

5043 28


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
288,Color,James Cameron,210.0,153.0,0.0,539.0,Jenette Goldstein,780.0,204843350.0,Action|Sci-Fi,...,983.0,English,USA,R,102000000.0,1991.0,604.0,8.5,2.35,13000
3969,Color,Sidney Lumet,161.0,121.0,0.0,682.0,Faye Dunaway,3000.0,NaN,Drama|Romance,...,317.0,English,USA,R,3800000.0,1976.0,977.0,8.1,1.85,17000
748,Color,Todd Phillips,173.0,101.0,480.0,767.0,Carmen Electra,881.0,88200225.0,Comedy|Crime,...,308.0,English,USA,PG-13,60000000.0,2004.0,869.0,6.1,2.35,0
2380,Color,Michael Dowse,136.0,97.0,31.0,562.0,Michael Biehn,2000.0,6923891.0,Comedy|Drama|Romance,...,83.0,English,USA,R,23000000.0,2011.0,2000.0,6.3,2.35,0
3776,Color,Michael Polish,12.0,121.0,35.0,473.0,Bobby Batson,4000.0,4700361.0,Drama,...,29.0,English,USA,PG-13,5000000.0,2015.0,849.0,4.6,2.35,0


In [4]:
for col in df.columns.sort_values():
    miss = 100 * (1 - df[col].count()/n)
    print('{:>30}: {}%'.format(col, miss))

        actor_1_facebook_likes: 0.13880626611144642%
                  actor_1_name: 0.13880626611144642%
        actor_2_facebook_likes: 0.25778306563553066%
                  actor_2_name: 0.25778306563553066%
        actor_3_facebook_likes: 0.4560777315090192%
                  actor_3_name: 0.4560777315090192%
                  aspect_ratio: 6.52389450723776%
                        budget: 9.756097560975608%
     cast_total_facebook_likes: 0.0%
                         color: 0.376759865159626%
                content_rating: 6.008328375966688%
                       country: 0.09914733293674427%
       director_facebook_likes: 2.0622645250842786%
                 director_name: 2.0622645250842786%
                      duration: 0.2974419988102328%
          facenumber_in_poster: 0.25778306563553066%
                        genres: 0.0%
                         gross: 17.52924846321634%
                    imdb_score: 0.0%
                      language: 0.2379535990481907%
     

### Custom database
We start with simple variables:

In [18]:
keepVar = ['movie_title',
           'actor_1_name','actor_2_name','actor_3_name',
           'duration']
wdf = df[keepVar].copy()
wdf.sample(5)

,movie_title,actor_1_name,actor_2_name,actor_3_name,duration
4675,Iraq for Sale: The War Profiteers,Scott Helvenston,Katy Helvenston-Wettengal,Donna Zovko,75.0
3910,3 Strikes,Mo'Nique,Mike Epps,Faizon Love,82.0
4190,Before Sunrise,Hanno Pöschl,Hans Weingartner,Dominik Castell,105.0
888,Burlesque,Eric Dane,Peter Gallagher,David Walton,119.0
4455,The Blue Room,Mathieu Amalric,Léa Drucker,Laurent Poitrenaux,76.0


Needed post-processing:
- lowerise all text to avoid missing doublons
- doublons/approximative doublons in `movie_title`,`actor_N_name`
- try one-hot encode the three `actor_N_name` (ignore order)


Notes:
- film titles (at least) have `\xa0` unbreakable spaces in the end: don't forget it when lookinf for titles in code!

In [19]:
# lowerise

toLowerVar = ('movie_title','actor_1_name','actor_2_name','actor_3_name')
for col in toLowerVar:
    wdf[col] = wdf[col].str.lower()
wdf.sample(5)

,movie_title,actor_1_name,actor_2_name,actor_3_name,duration
626,sky captain and the world of tomorrow,angelina jolie pitt,laurence olivier,bai ling,106.0
2543,road trip,ethan suplee,rachel blanchard,paulo costanzo,94.0
2571,my stepmother is an alien,jon lovitz,alyson hannigan,tony jay,105.0
2260,21 grams,naomi watts,eddie marsan,danny huston,124.0
3559,dressed to kill,angie dickinson,david margulies,nancy allen,104.0


In [20]:
# remove pure duplicates

print(wdf.duplicated().sum())
# wdf.loc[wdf.duplicated()].sort_values(by='movie_title')
wdf.drop_duplicates(inplace=True)

123


In [21]:
# look specifically at the rows where only film title is duplicated

print(wdf['movie_title'].duplicated(keep=False).sum())
wdf.loc[wdf['movie_title'].duplicated(keep=False)].sort_values(by='movie_title')

6


,movie_title,actor_1_name,actor_2_name,actor_3_name,duration
1852,brothers,natalie portman,jake gyllenhaal,bailee madison,105.0
2882,brothers,natalie portman,jake gyllenhaal,ethan suplee,105.0
3791,out of the blue,raymond burr,don gordon,jim byrnes,94.0
3840,out of the blue,william kircher,matthew sunderland,paul glover,100.0
1002,the host,j.d. evermore,chandler canterbury,rachel roberts,125.0
2988,the host,doona bae,kang-ho song,ah-sung ko,110.0


In [22]:
# no obvious error: don't want to check which one is correct... just drop them

wdf.drop_duplicates(subset='movie_title',inplace=True)
wdf.sample(5)

,movie_title,actor_1_name,actor_2_name,actor_3_name,duration
351,percy jackson & the olympians: the lightning t...,logan lerman,rosario dawson,steve coogan,118.0
2758,sarah's key,kristin scott thomas,aidan quinn,arben bajraktaraj,111.0
2251,school for scoundrels,jon heder,sarah silverman,jacinda barrett,108.0
4231,do you believe?,alexa penavega,delroy lindo,madison pettis,120.0
480,fantasia 2000,quincy jones,penn jillette,russi taylor,74.0


In [23]:
# beware: some films do not have all actors filled. Remove them.

actorVar = ('actor_1_name','actor_2_name','actor_3_name')
wdf = wdf.dropna(subset=actorVar,how='any')
wdf.shape

(4894, 5)

In [25]:
# same thing for duration

wdf = wdf.dropna(subset=['duration'],how='any')
wdf.shape

(4881, 5)

In [26]:
wdf.sample(5)

,movie_title,actor_1_name,actor_2_name,actor_3_name,duration
2298,connie and carla,debbie reynolds,nia vardalos,dash mihok,108.0
1721,doa: dead or alive,steve howey,holly valance,sarah carter,87.0
3681,harvard man,sarah michelle gellar,eric stoltz,joey lauren adams,99.0
1526,big momma's house,nia long,cedric the entertainer,tichina arnold,99.0
4954,the work and the story,richard moll,frank gerrish,christopher robin miller,77.0


### Dump the table
Note:
- we keep the index as we will use this to query films in the API

In [27]:
path,nameext= os.path.split(pathToData)
name,ext = os.path.splitext(nameext)

wdf.to_csv(os.path.join(path,name+'_CLEAN'+ext),index=True)